In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN, Bidirectional
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

Using TensorFlow backend.


In [2]:
traindata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_training-set.csv')
testdata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_testing-set.csv')

In [3]:
print(traindata['attack_cat'].value_counts())

Normal            56000
Generic           40000
Exploits          33393
Fuzzers           18184
DoS               12264
Reconnaissance    10491
Analysis           2000
Backdoor           1746
Shellcode          1133
Worms               130
Name: attack_cat, dtype: int64


In [4]:
traindata.head(5)

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [5]:
traindata=traindata.drop('id',axis=1)
testdata=testdata.drop('id',axis=1)

In [6]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

traindata['proto'] = lb_make.fit_transform(traindata['proto'])
traindata['service'] = lb_make.fit_transform(traindata['service'])
traindata['state'] = lb_make.fit_transform(traindata['state'])
traindata['attack_cat'] = lb_make.fit_transform(traindata['attack_cat'])

testdata['proto'] = lb_make.fit_transform(testdata['proto'])
testdata['service'] = lb_make.fit_transform(testdata['service'])
testdata['state'] = lb_make.fit_transform(testdata['state'])
testdata['attack_cat'] = lb_make.fit_transform(testdata['attack_cat'])

In [7]:
X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,-2]
x = testdata.iloc[:,1:42]
y = testdata.iloc[:,-2]

In [8]:
scaler = Normalizer().fit(X)
X = scaler.transform(X)
scaler = Normalizer().fit(x)
x = scaler.transform(x)

In [9]:
Y = np.array(Y)
y = np.array(y)
Y = to_categorical(Y)
y = to_categorical(y)
print (y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
X = np.reshape(X, (X.shape[0],X.shape[1],1))
x = np.reshape(x, (x.shape[0],x.shape[1],1))

print (X.shape)
print (Y.shape)

(175341, 41, 1)
(175341, 10)


In [12]:
folds=10

In [13]:
lstm_output_size = 70

cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(41, 1)))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(Bidirectional(LSTM(lstm_output_size)))
cnn.add(Dropout(0.1))
cnn.add(Dense(10, activation="softmax"))

cnn.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same", activation="relu", input_shape=(41, 1))`
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  """


In [ ]:
a=[]
for i in range(folds):
    print("Training on Fold: ",i+1)
    XTrain, XVal, yTrain, yVal = train_test_split(X, Y, test_size=0.2, random_state = np.random.randint(1,1000, 1)[0])
    cnn.fit(XTrain, yTrain, nb_epoch=200, validation_data=(XVal, yVal))
    scores = cnn.evaluate(XVal, yVal, verbose=0)
    print("%s: %.2f%%" % (cnn.metrics_names[1], scores[1]*100))
    a.append(scores[1] * 100)
    print("======="*12, end="\n\n\n")

Training on Fold:  1


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 140272 samples, validate on 35069 samples
Epoch 1/200
140272/140272 [==============================] - 183s 1ms/step - loss: 1.4010 - accuracy: 0.5247 - val_loss: 1.3717 - val_accuracy: 0.5307
Epoch 2/200
140272/140272 [==============================] - 204s 1ms/step - loss: 1.3625 - accuracy: 0.5325 - val_loss: 1.3347 - val_accuracy: 0.5319
Epoch 3/200
140272/140272 [==============================] - 203s 1ms/step - loss: 1.3328 - accuracy: 0.5437 - val_loss: 1.3176 - val_accuracy: 0.5480
Epoch 4/200
140272/140272 [==============================] - 203s 1ms/step - loss: 1.3252 - accuracy: 0.5471 - val_loss: 1.3199 - val_accuracy: 0.5523
Epoch 5/200
140272/140272 [==============================] - 203s 1ms/step - loss: 1.3185 - accuracy: 0.5492 - val_loss: 1.3437 - val_accuracy: 0.5433
Epoch 6/200
140272/140272 [==============================] - 204s 1ms/step - loss: 1.3130 - accuracy: 0.5505 - val_loss: 1.3115 - val_accuracy: 0.5521
Epoch 7/200
140272/140272 [================

Epoch 108/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6626 - accuracy: 0.7400 - val_loss: 0.6542 - val_accuracy: 0.7470
Epoch 109/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6612 - accuracy: 0.7407 - val_loss: 0.6579 - val_accuracy: 0.7436
Epoch 110/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6621 - accuracy: 0.7407 - val_loss: 0.6774 - val_accuracy: 0.7396
Epoch 111/200
140272/140272 [==============================] - 197s 1ms/step - loss: 0.6589 - accuracy: 0.7422 - val_loss: 0.6742 - val_accuracy: 0.7398
Epoch 112/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6603 - accuracy: 0.7415 - val_loss: 1.4661 - val_accuracy: 0.4404
Epoch 113/200
140272/140272 [==============================] - 197s 1ms/step - loss: 0.6590 - accuracy: 0.7421 - val_loss: 0.6633 - val_accuracy: 0.7417
Epoch 114/200
140272/140272 [==============================] - 196s 1ms/step - los

140272/140272 [==============================] - 196s 1ms/step - loss: 0.6271 - accuracy: 0.7549 - val_loss: 0.6296 - val_accuracy: 0.7546
Epoch 14/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6192 - accuracy: 0.7572 - val_loss: 0.6272 - val_accuracy: 0.7576
Epoch 15/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6176 - accuracy: 0.7584 - val_loss: 0.6281 - val_accuracy: 0.7563
Epoch 16/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6711 - accuracy: 0.7403 - val_loss: 0.9844 - val_accuracy: 0.6130
Epoch 17/200
140272/140272 [==============================] - 195s 1ms/step - loss: 0.8785 - accuracy: 0.6564 - val_loss: 0.8714 - val_accuracy: 0.6605
Epoch 18/200
140272/140272 [==============================] - 195s 1ms/step - loss: 0.8101 - accuracy: 0.6839 - val_loss: 0.7078 - val_accuracy: 0.7191
Epoch 19/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6417 - accuracy

...

140272/140272 [==============================] - 196s 1ms/step - loss: 0.5711 - accuracy: 0.7749 - val_loss: 0.6032 - val_accuracy: 0.7664
Epoch 147/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5673 - accuracy: 0.7762 - val_loss: 0.6138 - val_accuracy: 0.7636
Epoch 148/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5646 - accuracy: 0.7777 - val_loss: 0.6066 - val_accuracy: 0.7659
Epoch 149/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5654 - accuracy: 0.7770 - val_loss: 0.6149 - val_accuracy: 0.7651
Epoch 150/200
140272/140272 [==============================] - 195s 1ms/step - loss: 0.5681 - accuracy: 0.7750 - val_loss: 0.6053 - val_accuracy: 0.7666
Epoch 151/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5645 - accuracy: 0.7778 - val_loss: 0.6118 - val_accuracy: 0.7631
Epoch 152/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5681 - ac

Epoch 52/200
140272/140272 [==============================] - 197s 1ms/step - loss: 0.5858 - accuracy: 0.7701 - val_loss: 0.6060 - val_accuracy: 0.7683
Epoch 53/200
140272/140272 [==============================] - 197s 1ms/step - loss: 0.5863 - accuracy: 0.7696 - val_loss: 0.6067 - val_accuracy: 0.7669
Epoch 54/200
140272/140272 [==============================] - 197s 1ms/step - loss: 0.5850 - accuracy: 0.7704 - val_loss: 0.6174 - val_accuracy: 0.7645
Epoch 55/200
140272/140272 [==============================] - 198s 1ms/step - loss: 0.5857 - accuracy: 0.7703 - val_loss: 0.6187 - val_accuracy: 0.7657
Epoch 56/200
140272/140272 [==============================] - 198s 1ms/step - loss: 0.5838 - accuracy: 0.7707 - val_loss: 0.6048 - val_accuracy: 0.7703
Epoch 57/200
140272/140272 [==============================] - 197s 1ms/step - loss: 0.7016 - accuracy: 0.7300 - val_loss: 0.8702 - val_accuracy: 0.6634
Epoch 58/200
140272/140272 [==============================] - 198s 1ms/step - loss: 0.70

Epoch 106/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6680 - accuracy: 0.7412 - val_loss: 0.6633 - val_accuracy: 0.7442
Epoch 107/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6648 - accuracy: 0.7428 - val_loss: 0.6566 - val_accuracy: 0.7469
Epoch 108/200
140272/140272 [==============================] - 195s 1ms/step - loss: 0.6596 - accuracy: 0.7446 - val_loss: 0.6547 - val_accuracy: 0.7472
Epoch 109/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6556 - accuracy: 0.7459 - val_loss: 0.6569 - val_accuracy: 0.7481
Epoch 110/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6523 - accuracy: 0.7476 - val_loss: 0.6542 - val_accuracy: 0.7474
Epoch 111/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.6499 - accuracy: 0.7475 - val_loss: 0.6501 - val_accuracy: 0.7503
Epoch 112/200
140272/140272 [==============================] - 197s 1ms/step - los

140272/140272 [==============================] - 196s 1ms/step - loss: 0.5871 - accuracy: 0.7695 - val_loss: 0.5900 - val_accuracy: 0.7706
Epoch 12/200
140272/140272 [==============================] - 195s 1ms/step - loss: 0.5883 - accuracy: 0.7685 - val_loss: 0.5869 - val_accuracy: 0.7720
Epoch 13/200
140272/140272 [==============================] - 195s 1ms/step - loss: 0.5870 - accuracy: 0.7703 - val_loss: 0.5906 - val_accuracy: 0.7718
Epoch 14/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5862 - accuracy: 0.7691 - val_loss: 0.5878 - val_accuracy: 0.7715
Epoch 15/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5854 - accuracy: 0.7696 - val_loss: 0.5972 - val_accuracy: 0.7717
Epoch 16/200
140272/140272 [==============================] - 195s 1ms/step - loss: 0.5849 - accuracy: 0.7705 - val_loss: 0.5978 - val_accuracy: 0.7697
Epoch 17/200
140272/140272 [==============================] - 197s 1ms/step - loss: 0.5859 - accuracy

Epoch 65/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5719 - accuracy: 0.7739 - val_loss: 0.6116 - val_accuracy: 0.7650
Epoch 66/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5723 - accuracy: 0.7737 - val_loss: 0.6072 - val_accuracy: 0.7707
Epoch 67/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5775 - accuracy: 0.7721 - val_loss: 0.6304 - val_accuracy: 0.7586
Epoch 68/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5949 - accuracy: 0.7682 - val_loss: 0.6256 - val_accuracy: 0.7596
Epoch 69/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5809 - accuracy: 0.7723 - val_loss: 0.6013 - val_accuracy: 0.7697
Epoch 70/200
140272/140272 [==============================] - 196s 1ms/step - loss: 0.5732 - accuracy: 0.7735 - val_loss: 0.6136 - val_accuracy: 0.7667
Epoch 71/200
140272/140272 [==============================] - 197s 1ms/step - loss: 0.57